In [170]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3 as lite
import os
import sys
import matplotlib.pyplot as plt

In [171]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'}

In [172]:
cities = {'tx' : "austin"
          , 'ca' : 'los-angeles'
          , 'ga' : 'atlanta'
          , 'ma' : 'boston'
          , 'nc' : 'charlotte'
          , 'tx' : 'dallas'
          , 'co' : 'denver'
          , 'tx' : 'houston'
          , 'fl' : 'miami'
          , 'mn' : 'minneapolis'
          , 'ny' : 'new-york'
          , 'pa' : 'philadelphia'
          , 'az' : 'phoenix'
          , 'or' : 'portland'
          , 'ca' : 'san-francisco'
          , 'ca' : 'san-diego'
          , 'wa' : 'seattle'
          , 'dc' : 'washington'
         }

In [173]:
# CRAWL ALL THE PAGES OF INTERST

datadir = 'crawl/'
if not(os.path.isdir(datadir)):
    os.makedirs(datadir)

for k,v in cities.iteritems():
    running = True
    page = 1
    print ''
    print k,v
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        filename = datadir + k + '-' + v + '-' + str(page) + '.htm'
        if not(os.path.isfile(filename)):
            sys.stdout.write('-')
            r = requests.get(url, headers=headers)
            time.sleep(1)
            f = open(filename, 'w')
            f.write(r.text.encode('ascii', 'replace'))
            f.close()
            data = r.text
        else:
            sys.stdout.write('.')
            f = open(filename, 'r')
            data = f.read()
            f.close()
        soup = BeautifulSoup(data)
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
        page+=1



mn minneapolis
.......
ma boston
.............
wa seattle
..........
tx houston
...........
ca san-diego
............
nc charlotte
.......
dc washington
...........................
ny new-york
...............................................................
pa philadelphia
............
ga atlanta
............
co denver
...........
az phoenix
.........
fl miami
.........
or portland
..........

In [130]:
# Data extraction phase

times = []
fees = []
reviews = []
repeats = []
city = []
service = []
rating = []



for src in os.listdir(datadir):
    filename = datadir + src
    print 'Processing: ' + filename
    f = open(filename, 'r')
    data = f.read()
    f.close()
    soup = BeautifulSoup(data)
    sitters = soup.findAll('div', {'class': 'card-content'})
    for sitter in sitters:
        rtwrap = sitter.findAll('div', {'class': 'icon-wrapper'})
        if len(rtwrap) > 0:
            response_time = rtwrap[0].text.strip('\n')[19:]
        else:
            response_time = -1
        fee = sitter.findAll('div', {'class': 'price price--primary u-text-center'})
        review = sitter.find('span', {'class': 'vcard-review'})
        if review is not None and len(review) > 0:
            review = int(review.text.replace('Reviews', '').replace('Review', '').strip())
        else:
            review = 0
        repeat = sitter.findAll('div', {'class': 'icon-wrapper'})
        if len(repeat) == 2:
            repeat = repeat[1].text.strip('\n')[1]
        else:
            repeat = 0
        services = sitter.findAll('div', {'class': 'vcard-line-item'})
        full_star = sitter.findAll('i', {'class': 'ck-star'})
        ratings = len(full_star)
        half_star = sitter.findAll('i', {'class': 'ck-star-half'})
        if len(half_star) > 0:
            ratings += .5
        fees.append(fee[0].text.strip('\n$')[:2])
        city.append(v)
        times.append(response_time)
        reviews.append(review)
        repeats.append(repeat)
        service.append(services[-1].text.strip('\n'))
        rating.append(ratings)

df = pd.DataFrame({'city': city, 'fee': fees, 'response_time': times, 'review' : reviews, 'repeat' : repeats, 'services' : service, 'ratings': rating})

Processing: crawl/az-phoenix-1.htm
Processing: crawl/az-phoenix-2.htm
Processing: crawl/az-phoenix-3.htm
Processing: crawl/az-phoenix-4.htm
Processing: crawl/az-phoenix-5.htm
Processing: crawl/az-phoenix-6.htm
Processing: crawl/az-phoenix-7.htm
Processing: crawl/az-phoenix-8.htm
Processing: crawl/az-phoenix-9.htm
Processing: crawl/ca-san-diego-1.htm
Processing: crawl/ca-san-diego-10.htm
Processing: crawl/ca-san-diego-11.htm
Processing: crawl/ca-san-diego-12.htm
Processing: crawl/ca-san-diego-2.htm
Processing: crawl/ca-san-diego-3.htm
Processing: crawl/ca-san-diego-4.htm
Processing: crawl/ca-san-diego-5.htm
Processing: crawl/ca-san-diego-6.htm
Processing: crawl/ca-san-diego-7.htm
Processing: crawl/ca-san-diego-8.htm
Processing: crawl/ca-san-diego-9.htm
Processing: crawl/co-denver-1.htm
Processing: crawl/co-denver-10.htm
Processing: crawl/co-denver-11.htm
Processing: crawl/co-denver-2.htm
Processing: crawl/co-denver-3.htm
Processing: crawl/co-denver-4.htm
Processing: crawl/co-denver-5.ht

In [105]:
df.shape

(3098, 7)

In [131]:
df.head()

,city,fee,ratings,repeat,response_time,review,services
0,portland,35,5,8,Few Hours,26,Boarding and Daycare
1,portland,35,5,1,Day,30,"Boarding, Checkups and Visits, and Pick up and..."
2,portland,35,0,0,Few Hours,0,"Boarding, Daycare, Checkups and Visits, Walkin..."
3,portland,38,5,2,Few Days,3,"Boarding, Daycare, Checkups and Visits, Walkin..."
4,portland,25,5,1,Few Hours,5,"Boarding, Checkups and Visits, Walking, and Ba..."


In [39]:
# SAVE TO CSV FILE

df.to_csv('dog-vacay.csv', sep='\t')

In [132]:
#separate services
df['boarding'] = df['services'].map(lambda x: 1 if "Boarding" in x else 0)
df['sitting'] = df['services'].map(lambda x: 1 if "Sitting" in x else 0)
df['daycare'] = df['services'].map(lambda x: 1 if "Daycare" in x else 0)
df['checkups'] = df['services'].map(lambda x: 1 if "Checkups and Visits" in x else 0)
df['walking'] = df['services'].map(lambda x: 1 if "Walking" in x else 0)
df['bathing'] = df['services'].map(lambda x: 1 if "Bathing" in x else 0)
df['training'] = df['services'].map(lambda x: 1 if "Training" in x else 0)
df['grooming'] = df['services'].map(lambda x: 1 if "Grooming" in x else 0)
df['pickup/dropoff'] = df['services'].map(lambda x: 1 if "Pick up and Drop off" in x else 0)
df['services'] = 0

In [133]:
#response time
df['response_time'] = df['response_time'].map(lambda x: 1 if x == 'Few Hours                        ' else 2 if x == 'Day                        ' else 3)

In [134]:
df.head()

,city,fee,ratings,repeat,response_time,review,services,boarding,sitting,daycare,checkups,walking,bathing,training,grooming,pickup/dropoff
0,portland,35,5,8,1,26,0,1,0,1,0,0,0,0,0,0
1,portland,35,5,1,2,30,0,1,0,0,1,0,0,0,0,1
2,portland,35,0,0,1,0,0,1,0,1,1,1,0,0,0,0
3,portland,38,5,2,3,3,0,1,0,1,1,1,0,0,0,0
4,portland,25,5,1,1,5,0,1,0,0,1,1,1,0,0,0


In [167]:
df.describe()

,fee,ratings,repeat,response_time,review,services,boarding,sitting,daycare,checkups,walking,bathing,training,grooming,pickup/dropoff
count,3098.000000,3098.000000,3098.000000,3098.000000,3098.000000,3098,3098,3098.000000,3098.000000,3098.000000,3098.000000,3098.000000,3098.000000,3098.000000,3098.000000
mean,38.003228,3.504358,1.738218,1.582311,9.620400,0,1,0.293738,0.640736,0.406068,0.381214,0.149128,0.023564,0.008715,0.194642
std,10.882084,2.272712,2.239509,0.760570,17.783667,0,0,0.455547,0.479862,0.491177,0.485763,0.356273,0.151710,0.092963,0.395989
min,10.000000,0.000000,0.000000,1.000000,0.000000,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,0.000000,1.000000,0.000000,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,35.000000,5.000000,1.000000,1.000000,3.000000,0,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,45.000000,5.000000,3.000000,2.000000,11.000000,0,1,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,95.000000,5.000000,9.000000,3.000000,187.000000,0,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [136]:
#Create a scatter plot matrix
a = pd.scatter_matrix(df, alpha=.05, figsize=(10,10))
plt.show()

In [110]:
#s = df['response_time'][0]
#sentence = ''.join(s.split())
#sentence

u'FewHours'

In [150]:
#Median fee
df["fee"].median()

35.0

In [152]:
#Histogram of fees
df["repeat"] = df["repeat"].astype(float)
p = df['fee'].hist()
plt.show()

In [153]:
#Histogram of repeat guests
df["repeat"] = df["repeat"].astype(float)
p = df['repeat'].hist()
plt.show()

In [174]:
df["review"] = df["review"].astype(float)
p = df['review'].hist()
plt.show()

In [161]:
df.groupby('city').sum()

,fee,ratings,repeat,response_time,review,services,boarding,sitting,daycare,checkups,walking,bathing,training,grooming,pickup/dropoff
city,,,,,,,,,,,,,,,
portland,117734,10856.5,5385,4902,29804,0,3098,910,1985,1258,1181,462,73,27,603


In [169]:
df['city']

0     portland
1     portland
2     portland
3     portland
4     portland
5     portland
6     portland
7     portland
8     portland
9     portland
10    portland
11    portland
12    portland
13    portland
14    portland
...
3083    portland
3084    portland
3085    portland
3086    portland
3087    portland
3088    portland
3089    portland
3090    portland
3091    portland
3092    portland
3093    portland
3094    portland
3095    portland
3096    portland
3097    portland
Name: city, Length: 3098, dtype: object